## Step 1: go to the right folder

In [14]:
import os
%pwd

'/Users/krystalli/Documents/End-to-End-Wine-Quality-Prediction'

In [2]:
os.chdir("../")
%pwd

'/Users/krystalli/Documents/End-to-End-Wine-Quality-Prediction'

## Step 2: update files according to workflow (in README)

### 2.1 Update config.yaml (set up parameters for data ingestion)
### 2.2 Update schema.yaml, params.yaml
### 2.3 Update entity


In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass (frozen=True)
# meaning the instances of the class are immutable. Once an instance is created, 
# you cannot modify its attributes. This is similar to creating a read-only object.

class DataIngestionConfig: # not python class, but a dataclass!
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

### 2.4 Update the configuration manager (in src config)

In src/constants constructor file, return all the yaml path. 

Then import all of them to create config mgr class

In [16]:
from wine_quality_prediction_project.constants import *
from wine_quality_prediction_project.utils.common import read_yaml, create_directories

In [17]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    # get all the data from the config file
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

### 2.5 Update the components

In [18]:
import os
import urllib.request as request
import zipfile
from wine_quality_prediction_project import logger
from wine_quality_prediction_project.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

### 2.6 Update pipeline

In [19]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-11-17 15:28:12,817: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-11-17 15:28:12,819: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-17 15:28:12,820: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-11-17 15:28:12,821: INFO: common: created directory at: artifacts]
[2024-11-17 15:28:12,822: INFO: common: created directory at: artifacts/data_ingestion]
[2024-11-17 15:28:13,743: INFO: 3349882613: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 26112
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "2d987e9f6e46bf7d2bc829dc2a7c30c6ab58cbaa0a255ed7247423d1c53741df"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 435E:10291E:3ABFEB:40CE3B:673A7C0C
Accept-Ranges: bytes
Date: Sun, 17